In [1]:
import os
import torch
import monai
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, Orientationd, ScaleIntensityRanged, CropForegroundd,
    RandCropByPosNegLabeld, RandFlipd, RandRotate90d, RandShiftIntensityd, EnsureTyped, DivisiblePadd,ResizeWithPadOrCropd
)
from monai.data import DataLoader, CacheDataset
from monai.networks.nets import SwinUNETR
from monai.utils import set_determinism
from monai.data.image_reader import NibabelReader
from sklearn.metrics import mean_absolute_error, mean_squared_error,accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import re

In [2]:
 # Define directories
train_path = 'dataset/MICCAI_BraTS2020_TrainingData/'
# val_path = 'dataset/MICCAI_BraTS2020_ValidationData/'
modality_keys = ["flair", "t1ce", "t2"]
in_channels = len(modality_keys)
out_channels = len(modality_keys) 

In [3]:
def create_data_list(data_dir, patient_ids, lables, modality_keys):
    data_list = []
    for idx, patient in enumerate(patient_ids):
        patient_dir = f"{data_dir}{patient}/"
        if os.path.isdir(patient_dir):
            data_dict = {key: os.path.join(patient_dir, f"{patient}_{key}.nii") for key in modality_keys}
            data_dict['lable'] = lables[idx]
            data_list.append(data_dict)

    return data_list

In [4]:
def preprocess_labels(csv_file_path):
    df = pd.read_csv(csv_file_path)
    
    def extract_number(value):
        if isinstance(value, str):
            match = re.search(r'\d+', value)
            return int(match.group()) if match else None
        return value

    df['Survival_days'] = df['Survival_days'].apply(extract_number)
    df['Survival_days'] = pd.to_numeric(df['Survival_days'], errors='coerce')
    df = df.dropna(subset=['Survival_days'])
    df['Survival_days'] = df['Survival_days'].astype(int)
    
    return df['Survival_days'].values, df['Brats20ID'].values

In [5]:
# def preprocess_labels_2(csv_file_path):
#     df = pd.read_csv(csv_file_path)
#     df['Survival_days'] = pd.to_numeric(df['Survival_days'], errors='coerce')
#     df = df.dropna(subset=['Survival_days'])
#     df['Survival_days'] = df['Survival_days'].astype(int)
#     df['Survival_class'] = df['Survival_days'].apply(lambda x: 0 if x < 300 else (1 if x < 450 else 2))
#     return df['Survival_class'].values, df['Brats20ID'].values

In [6]:
all_labels, all_data_ids = preprocess_labels(f'{train_path}/survival_info.csv')
train_id, val_id, train_labels, val_labels = train_test_split(all_data_ids, all_labels, test_size=0.2, random_state=42)


In [7]:

train_data_list = create_data_list(train_path, train_id, train_labels, modality_keys)
val_data_list = create_data_list(train_path, val_id, val_labels, modality_keys)


# since we dont have lables for validation dataset not using it for validation
# val_data_list = create_data_list(val_path, modality_keys)
# val_labels = preprocess_labels(f'{val_path}/survival_evaluation.csv')



In [8]:
import nibabel as nib
import torch

# Load the NIfTI image
file_path = train_data_list[0]['flair']
img = nib.load(file_path)

# Get the image data as a numpy array   
img_data = img.get_fdata()

# Convert the numpy array to a PyTorch tensor
img_tensor = torch.tensor(img_data, dtype=torch.float32)

# Print the shape of the tensor
print(f'Tensor shape: {img_tensor.shape}')


Tensor shape: torch.Size([240, 240, 155])


In [9]:
def get_transforms(modality_keys, pixdim=(1.0, 1.0, 1.0), is_train=True):
    transform_list = [
        LoadImaged(keys=modality_keys, reader=NibabelReader()),
        EnsureChannelFirstd(keys=modality_keys),
        Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
        Orientationd(keys=modality_keys, axcodes="RAS"),
        ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
        ResizeWithPadOrCropd(keys=modality_keys, spatial_size=(256, 256, 160)),  # Kept original size
        EnsureTyped(keys=modality_keys),
    ]
    
    if is_train:
        transform_list.extend([
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=1),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=2),
            RandRotate90d(keys=modality_keys, prob=0.5, max_k=3),
            RandShiftIntensityd(keys=modality_keys, offsets=0.10, prob=0.5),
        ])
    
    return Compose(transform_list)

In [10]:
# def get_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0), spatial_size=(64, 64, 64)):
#     transforms = Compose(
#         [
#             LoadImaged(keys=modality_keys, reader=NibabelReader()),
#             EnsureChannelFirstd(keys=modality_keys),
#             Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
#             Orientationd(keys=modality_keys, axcodes="RAS"),
#             ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
#             CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
#             DivisiblePadd(keys=modality_keys, k=32),
#             RandCropByPosNegLabeld(keys=modality_keys, label_key=modality_keys[0], spatial_size=spatial_size, pos=1, neg=1, num_samples=1, image_key=modality_keys[0], image_threshold=0),
#             RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=0),
#             RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=1),
#             RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=2),
#             RandRotate90d(keys=modality_keys, prob=0.5, max_k=3),
#             RandShiftIntensityd(keys=modality_keys, offsets=0.10, prob=0.5),
#             EnsureTyped(keys=modality_keys),
#         ]
#     )
#     return transforms

In [11]:
# def get_val_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0)):
#     transforms = Compose(
#         [
#             LoadImaged(keys=modality_keys, reader=NibabelReader()),
#             EnsureChannelFirstd(keys=modality_keys),
#             Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
#             Orientationd(keys=modality_keys, axcodes="RAS"),
#             ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
#             CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
#             DivisiblePadd(keys=modality_keys, k=32),
#             EnsureTyped(keys=modality_keys),
#         ]
#     )
#     return transforms

In [12]:
train_transforms = get_transforms(modality_keys, is_train=True)
val_transforms = get_transforms(modality_keys, is_train=False)


In [13]:
train_ds = CacheDataset(
    data=train_data_list,
    transform=train_transforms,
    cache_rate=0.5,
    num_workers=4,
)
train_loader = DataLoader(train_ds,  batch_size=2, shuffle=True, num_workers=4)


Loading dataset:   0%|          | 0/94 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 94/94 [00:31<00:00,  3.00it/s]


In [14]:
# for data in train_loader:
#     print(data['flair'].shape)

In [15]:
val_ds = CacheDataset(
    data=val_data_list,
    transform=val_transforms,
    cache_rate=0.5,
    num_workers=4,
)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=2)

Loading dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 24/24 [00:07<00:00,  3.40it/s]


In [16]:
# Load the pretrained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
model = SwinUNETR(
    img_size=(256, 256, 160),  # Kept original size
    in_channels=len(modality_keys),
    out_channels=len(modality_keys),
    feature_size=24,  # Reduced from 48 to save memory
    use_checkpoint=True,
).to(device)

# Load the saved weights
modality_used = "_".join(modality_keys)
model_save_path = f"model_saved/swin_unetr_{modality_used}_best.pth"

model.load_state_dict(torch.load(model_save_path))
model.eval()

SwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 24, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=24, out_features=72, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=24, out_features=24, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=24, out_features=96, bias=True)
              (linear2): Linear(in_features=

In [17]:
# # 2. Modify the SwinUNETR class to add a method for feature extraction
# class FeatureExtractorSwinUNETR(SwinUNETR):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
    
#     def extract_features(self, x):
#         print(" at start",x.size())
#         hidden_states = self.swinViT(x, self.normalize)
#         enc0 = self.encoder1(x)
#         print(" after enco 1",x.size())
#         enc1 = self.encoder2(hidden_states[0])
#         print(" after enco 2",enc1.size())
#         enc2 = self.encoder3(hidden_states[1])
#         print(" after enco 3",enc2.size())
#         enc3 = self.encoder4(hidden_states[2])
#         print(" after enco 4",enc3.size())
#         # dec3 = self.encoder10(hidden_states[3])
#         # print(" after dec3",dec3.size())
#         # return torch.cat([enc0, enc1, enc2, enc3, dec3], dim=1)
#         return torch.cat([enc0, enc1, enc2, enc3], dim=1)

sizes of features at diffrent stages are as follows
- At 0 state torch.Size([2, 48, 32, 32, 32])
- At 1 state torch.Size([2, 96, 16, 16, 16])
- At 2 state torch.Size([2, 192, 8, 8, 8])
- At 3 state torch.Size([2, 384, 4, 4, 4])
- At 4 state torch.Size([2, 768, 2, 2, 2])

In [18]:
class FeatureExtractorSwinUNETR(SwinUNETR):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def extract_features(self, x):
        hidden_states = self.swinViT(x, self.normalize)
        
        # Adaptive pooling to resize all hidden states to a common size
        pooled_states = []
        for state in hidden_states:
            # Adaptive average pooling to 4x4x4
            pooled = F.adaptive_avg_pool3d(state, (4, 4, 4))
            pooled_states.append(pooled)
        
        # Concatenate the pooled states
        return torch.cat(pooled_states, dim=1)


In [20]:
#  Create an instance of the feature extractor
feature_extractor = FeatureExtractorSwinUNETR(
     img_size=(256, 256, 160),  # Kept original size
    in_channels=len(modality_keys),
    out_channels=len(modality_keys),
    feature_size=24,  # Reduced from 48 to save memory
    use_checkpoint=True,
).to(device)
feature_extractor.load_state_dict(model.state_dict())
feature_extractor.eval()

FeatureExtractorSwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 24, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=24, out_features=72, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=24, out_features=24, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=24, out_features=96, bias=True)
              (linear2): Lin

In [ ]:

# # 3. Extract features from your dataset
# def extract_features_from_dataset(dataset, feature_extractor):
#     features = []
#     labels = []
    
#     for data in dataset:
#         inputs = torch.cat([data[key] for key in modality_keys], dim=1).to(device)
#         with torch.no_grad():
#             feature = feature_extractor.extract_features(inputs)
        
#         # Assuming the feature is 5D (batch, channels, depth, height, width)
#         # We'll flatten it to 2D (batch, features)
#         feature = feature.view(feature.size(0), -1)
        
#         features.append(feature.cpu().numpy())
#         labels.append(data['label'].cpu().numpy())  # Assuming you have labels in your dataset
    
#     return np.concatenate(features), np.concatenate(labels)


In [21]:
def extract_features_from_dataset(dataloader, feature_extractor):
    features = []
    labels = []
    
    for batch in dataloader:
        inputs = torch.cat([batch[key] for key in modality_keys], dim=1).to(device)
        with torch.no_grad():
            feature = feature_extractor.extract_features(inputs)
        
        # Global average pooling to reduce spatial dimensions
        feature = torch.mean(feature, dim=[2, 3, 4])
        
        features.append(feature.cpu().numpy())
        # labels.append(batch['label'].cpu().numpy())  # Assuming you have labels in your dataset
    
    return np.concatenate(features)


In [22]:

# Extract features for training and validation sets
train_features = extract_features_from_dataset(train_loader, feature_extractor)
val_features = extract_features_from_dataset(val_loader, feature_extractor)


In [23]:
# Preprocess the data
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)


In [24]:
# Hyperparameter tuning for LightGBM
param_dist = {
    'num_leaves': [31, 63, 127],
    'max_depth': [-1, 5, 10, 20],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'min_child_samples': [10, 20, 30]
}

In [25]:
from sklearn.model_selection import RandomizedSearchCV

In [26]:
# Train LightGBM regression model
lgb_model = lgb.LGBMRegressor(random_state=42)
random_search = RandomizedSearchCV(lgb_model, param_distributions=param_dist, n_iter=20, cv=5, random_state=42, n_jobs=-1)
random_search.fit(train_features_scaled, train_labels)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025651 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38685
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024832 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38685
[LightGBM] [Info] Number of data points in the train set: 151, number of used features: 744
[LightGBM] [Info] Start training from score 454.927152
[LightGBM] [Info] Number of data points in the train set: 151, number of used features: 744
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022694 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38685
[LightGBM] [Info] Start training from score 454.927152
[LightGBM] [Info] Number of data points in the train set: 151, number of used features: 744
[LightGBM] [Info] Auto-

RandomizedSearchCV(cv=5, estimator=LGBMRegressor(random_state=42), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [-1, 5, 10, 20],
                                        'min_child_samples': [10, 20, 30],
                                        'n_estimators': [100, 200, 300],
                                        'num_leaves': [31, 63, 127]},
                   random_state=42)

In [27]:
# Make predictions
y_pred = random_search.predict(val_features)


In [28]:
# Evaluate the model
mae = mean_absolute_error(val_labels, y_pred)
rmse = mean_squared_error(val_labels, y_pred, squared=False)
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 345.34959720482783
Root Mean Squared Error: 413.62255275042423


/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
best_lgb_model = random_search.best_estimator_

# Create an ensemble
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

ensemble = VotingRegressor(
    estimators=[('lgb', best_lgb_model), ('rf', rf_model)]
)

In [30]:
# Fit the ensemble
ensemble.fit(train_features_scaled, train_labels)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004887 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 47872
[LightGBM] [Info] Number of data points in the train set: 188, number of used features: 744
[LightGBM] [Info] Start training from score 446.531915
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

VotingRegressor(estimators=[('lgb',
                             LGBMRegressor(learning_rate=0.01,
                                           random_state=42)),
                            ('rf', RandomForestRegressor(random_state=42))])

In [31]:
# Make predictions
y_pred = ensemble.predict(val_features)


In [32]:
# Evaluate the model
mae = mean_absolute_error(val_labels, y_pred)
rmse = mean_squared_error(val_labels, y_pred, squared=False)
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 372.083236102414
Root Mean Squared Error: 428.1846171933718


/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [33]:
train_features.shape


(188, 744)

In [34]:
val_features.shape

(48, 744)